Last modified in March 31, 2024 by Dabin Yang

Original code generated by Brown, N.K, 2022

# FEA_SOLVER_GENERAL

In [ ]:
import numpy as np
'Define the Function that creates the rectangular mesh'
from skimage.measure import label
def label_( x): return label(x, connectivity=1)
def get_zero_label(d, labels):
    for i in range(d.shape[0]):
        for j in range(d.shape[1]):
            if d[i,j]==0:
                return labels[i,j]

def isolate_largest_group_original(x):
    """Isolates the largest group. 
    this original version has problems with periodicity. The v2 version fixes this. """
    x_original = np.copy(x)    
    labels= label_(x)
    zerolabel=get_zero_label(x,labels)
    group_names = np.unique(labels)      
    group_names = [g for g in group_names if g!=zerolabel]   
    vols = np.array([(labels==name).sum() for name in group_names])
    largestgroup_name = group_names[np.argmax(vols)]    
    x = labels == largestgroup_name
    x = x.astype("int")
    design_ok = np.all(x==x_original)  
    return x, design_ok

def rectangularmesh(Lx,Ly,ElementsX,ElementsY):
    Nx=ElementsX
    Ny=ElementsY
    xx=np.linspace(0,Lx,ElementsX+1)
    yy=np.linspace(0,Ly,ElementsY+1)
    nodeCoor=np.zeros(((Nx+1)*(Ny+1),2))
    elementNodes=np.zeros((Nx*Ny,4))
    for j in range(0,len(yy)):
        for i in range(0,len(xx)):
            call=(Nx+1)*(j)+i
            nodeCoor[call,0]=xx[i]
            nodeCoor[call,1]=yy[j]     
    for j in range(0,Ny):
        for i in range(0,Nx):
            d=(Nx)*(j)+i;
            elementNodes[d,0]=(Nx+1)*(j)+i
            elementNodes[d,1]=(Nx+1)*(j)+i+1
            elementNodes[d,2]=(Nx+1)*(j+1)+i+1
            elementNodes[d,3]=(Nx+1)*(j+1)+i
    return nodeCoor, elementNodes

def shapeFunctionQ4(xi,eta):
    shape=(1/4)*np.matrix([[(1-xi)*(1-eta)],
                           [(1+xi)*(1-eta)],
                           [(1+xi)*(1+eta)],
                           [(1-xi)*(1+eta)]])
    naturalDeriv=(1/4)*np.matrix([[-(1-eta),-(1-xi)],
                                  [ (1-eta),-(1+xi)],
                                  [ (1+eta), (1+xi)],
                                  [-(1+eta), (1-xi)]])
    return(shape,naturalDeriv) 

#-----------------------------------------------
'Determine Jacobian Matrix and Inverse of Jacobian'
def Jacobian(nodeCoor,naturalDeriv):
    JacobianMatrix=np.transpose(nodeCoor)*naturalDeriv
    invJacobian=np.linalg.inv(JacobianMatrix)
    XYDerivative=naturalDeriv*invJacobian
    return(JacobianMatrix,invJacobian,XYDerivative)

'Calculation of the System Stiffness Matrix'
def formStiffness2D(GDof,TotElements,numberNodes,elementNodes,nodeCoor,materials,rho,thickness,VoidCheck):
    """Compute Stiffness Matrix and (mass matrix)
       for plane stress Q4 elements"""
    stiffness=np.zeros((GDof,GDof))

    mass=np.zeros((GDof,GDof))
    
    '2 by 2 quadrature'
    gaussLocations=np.array([[-.577350269189626, -.577350269189626],
                             [ .577350269189626, -.577350269189626],
                             [ .577350269189626,  .577350269189626],
                             [-.577350269189626,  .577350269189626]])
    gaussWeights=np.array([[1],[1],[1],[1]])

    for e in range(0,TotElements):
        indice=(elementNodes[e,:])
        indice=indice.astype(int)
        elementDof=[]
        elementDof=np.append(indice,indice+numberNodes)
        ndof=len(indice)
        'Shape Functions and Derivatives'
    
        'Cycle for Gauss Point'
        for q in range(0,len(gaussWeights)):
            GaussPoint=gaussLocations[q,:]
            xi=GaussPoint[0]
            eta=GaussPoint[1]
            #Determine Shape Functions and Derivatives 
            sFQ4=shapeFunctionQ4(xi,eta)
            naturalDeriv=sFQ4[1]
            
            #Determine Jacobian
            JR=Jacobian(nodeCoor[indice,:],naturalDeriv)
            JacobianMatrix=JR[0]
            
            #invJacobian=JR[1]
            XYderivative=JR[2]
              #------------------------------------------------
            'Create the B matrix'
            B=np.zeros((3,2*ndof))
            B[0,0:ndof]         =np.transpose(XYderivative[:,0])
            B[1,ndof:(2*ndof)]  =np.transpose(XYderivative[:,1])
            B[2,0:ndof]         =np.transpose(XYderivative[:,1])
            B[2,ndof:(2*ndof)]  =np.transpose(XYderivative[:,0])
        
              #-------------------------------------------------
            'Stiffness matrix'
            'Assign Unique Material Properties Depending on Void or Material'
            if VoidCheck[e]==1:
                E=materials[0,0]
                poisson=materials[0,1]
            else:
                E=materials[1,0]
                poisson=materials[1,1]
            C=np.matrix([[1,poisson,0], [poisson,1,0],[0,0,(1-poisson)/2]])*(E/(1-(poisson**2)))
            Mat1=np.asarray(np.transpose(B)*C*thickness*B*1*np.linalg.det(JacobianMatrix))
         

            stiffness[np.ix_(elementDof,elementDof)]=stiffness[np.ix_(elementDof,elementDof)]+Mat1
            
    return(stiffness,mass)
def solution(GDof,prescribedDof,stiffness,force):
    # Function to find solution in terms of global dispalcements
    activeDof=[]
    GDof_list=list(range(0,GDof))
    for i in GDof_list:
        if i not in prescribedDof[0]:
            activeDof.append(i)
    ActiveStiffness=np.zeros((len(activeDof),len(activeDof)))

    ActiveStiffness=stiffness[np.ix_(activeDof,activeDof)]
    ActiveForce=np.zeros((len(activeDof),1))
    for i in range(0,len(activeDof)):
        ActiveForce[i]=force[activeDof[i]]
    
    #U=cupy.matmul(cupy.linalg.inv(cupy.array(ActiveStiffness)),cupy.array(ActiveForce))
    #U=cupy.ndarray.get(U)
    U=np.matmul(np.linalg.inv(ActiveStiffness),ActiveForce)
    displacements=np.zeros((GDof,1))
    displacements[activeDof]=U
    return displacements
def stresses2D(GDof,TotElements,nodeCoor,numberNodes,displacements,UX,UY,materials,ScaleFactor,VoidCheck,elementNodes):    
        gaussLocations=np.array([[-.577350269189626, -.577350269189626],
                                 [ .577350269189626, -.577350269189626],
                                 [ .577350269189626,  .577350269189626],
                                 [-.577350269189626,  .577350269189626]])
        gaussWeights=np.array([[1],[1],[1],[1]])
        'stresses at nodes'
        stress=np.zeros((TotElements,len(elementNodes[0]),3))        
        for e in range(0,TotElements):
            indice=(elementNodes[e,:])
            indice=indice.astype(int)
            elementDof=[]
            elementDof=np.append(indice,indice+numberNodes)

            nn=len(indice)
            for q in range(0,len(gaussWeights)):
                pt=gaussLocations[q,:]
                xi=pt[0]
                eta=pt[1]
                sFQ4=shapeFunctionQ4(xi,eta)
                naturalDeriv=sFQ4[1]
                #Determine Jacobian
                JR=Jacobian(nodeCoor[indice,:],naturalDeriv)
                XYderivative=JR[2]
                'Create the B matrix'
                B=np.zeros((3,2*nn))
                B[0,0:nn]         =np.transpose(XYderivative[:,0])
                B[1,nn:(2*nn)]  =np.transpose(XYderivative[:,1])
                B[2,0:nn]         =np.transpose(XYderivative[:,1])
                B[2,nn:(2*nn)]  =np.transpose(XYderivative[:,0])

                'Element Deformation'
                #Check to See What Material Properties Should Be Assigned
                if VoidCheck[e]==1:
                    E=materials[0,0]
                    poisson=materials[0,1]
                else:
                    E=materials[1,0]
                    poisson=materials[1,1]
                C=np.matrix([[1,poisson,0], [poisson,1,0],[0,0,(1-poisson)/2]])*(E/(1-(poisson**2)))
                strain=np.matmul(B,displacements[elementDof])
                stress[e,q,:]=np.transpose(np.dot(C,strain))

        return(stress)
def FEASolve(VoidCheck,Lx,Ly,ElementsX,ElementsY,LC_Nodes,Loaded_Directions,BC_Nodes,Stress):
    'Input Linear Material Properties'
    materials=np.zeros((2,2))
    #Real Material Properties
    materials[0,0]=10e9 #Modulus of Elasticity
    materials[0,1]=0.3 #Poisson's Ratio
    
    #Void Material Representation 
    """Taken from: An FEM Analysis with Consideration of Random Void Defects for 
    Predicting the Mechanical Properties of 3D Braided Composities, Kun Xu and Xiaomei Qian"""
    materials[1,0]=1 #Modulus of Elasticity (1e-6 MPa)
    materials[1,1]=0.000001 #Poisson's Ratio
    
    'Input Desired Loading Condition'
    P=5e5#Magnitude
    
    'Mesh Generation'
    TotElements=ElementsX*ElementsY #Total Number of Elements 
    RectOutput=rectangularmesh(Lx,Ly,ElementsX,ElementsY)
    nodeCoor=(RectOutput[0])
    elementNodes=(RectOutput[1])
    xx=nodeCoor[:,0]
    yy=nodeCoor[:,1]
    numberNodes=len(xx)
    'Global Number of Degrees of Freedom'
    GDof=2*numberNodes

        #-------------------------------------------------------------------
    'Boundary Conditions'
    '''This portion can be adjusted in accordance with the necessary boundary
    conditions of the given problem'''
    
    'BC currently set up to limit displacement on Y-Axis with distributed force on X=Lx' 
    fixedNodeX=list(BC_Nodes)
    fixedNodeY=list(BC_Nodes+numberNodes)
 
    prescribedDof=[fixedNodeX+fixedNodeY]
    'Input Force Vectors'
    #Currently Assuming Distributed load applied at xx=Lx (Tensile Testing)
    force=np.zeros((GDof,1))
    
    
    #--------------------------------------------------------
    #|                                                      |
    #|                                                      |
    #|    Apply for Point Load on Bottom Right Corner       |
    #|                        |                             |
    #|                        v                             |
    #--------------------------------------------------------
    LD_Count=0
    for F in range(0,len(LC_Nodes)):
        if F/2==int(F/2) and F/2!=0:
            LD_Count+=1
        force[int(LC_Nodes[F])]=(P/2)*Loaded_Directions[LD_Count]
    FS2D=formStiffness2D(GDof,TotElements,numberNodes,elementNodes,nodeCoor,materials,1,1,VoidCheck)                                                         
    stiffness=FS2D[0]
    displacements=solution(GDof,prescribedDof,stiffness,force)
    UX=np.asarray(np.transpose(displacements[0:numberNodes]))
    UY=np.asarray(np.transpose(displacements[numberNodes:GDof]))
    MaxDisplacements=displacements.min()
    StrainEnergy=np.matmul(np.transpose(displacements),stiffness)
    StrainEnergy=np.matmul(StrainEnergy,displacements)
    RectOutput=rectangularmesh(Lx,Ly,ElementsX,ElementsY)

    if Stress is True:
        ScaleFactor=0.1      
        StressVal=stresses2D(GDof,TotElements,nodeCoor,numberNodes,displacements,UX,UY,materials,ScaleFactor,VoidCheck,elementNodes)
        stress=StressVal
        sigmax=stress[:,:,0]
        sigmay=stress[:,:,1]
        tauxy=stress[:,:,2]
    
        stressx_node=np.zeros((ElementsX+1,ElementsY+1))
        stressy_node=np.zeros((ElementsX+1,ElementsY+1))
        stressxy_node=np.zeros((ElementsX+1,ElementsY+1))
        stressx_element=np.zeros((ElementsX*ElementsY,1))
        stressy_element=np.zeros((ElementsX*ElementsY,1))
        stressxy_element=np.zeros((ElementsX*ElementsY,1))
    
        #stressmat=np.zeros(numberNodes)
        Count=0
        
        for j in range(0,ElementsY):
            for i in range(0,ElementsX):
                #Change sigmax to sigmay for different stress distribution
                stressx_node[np.ix_(range(i,i+2),range(j,j+2))]=np.reshape(sigmax[Count,:],(2,2))
                stressy_node[np.ix_(range(i,i+2),range(j,j+2))]=np.reshape(sigmay[Count,:],(2,2))
                stressxy_node[np.ix_(range(i,i+2),range(j,j+2))]=np.reshape(tauxy[Count,:],(2,2))
                Count=Count+1
        for ii in range(0,len(stressx_element)):
            stressx_element[ii]=np.mean(sigmax[ii,:])
            stressy_element[ii]=np.mean(sigmay[ii,:])
            stressxy_element[ii]=np.mean(tauxy[ii,:])
       
        stressx_element=np.reshape(stressx_element,(ElementsX,ElementsY))
        stressy_element=np.reshape(stressy_element,(ElementsX,ElementsY))
        stressxy_element=np.reshape(stressxy_element,(ElementsX,ElementsY))
               
        ''' Calculate the Von Mises Stress'''
        VonMises_element=np.sqrt((stressx_element*stressx_element)-(stressx_element*stressy_element)+(stressy_element*stressy_element)+(3*stressxy_element*stressxy_element))
        VonMises_node=np.sqrt((stressx_node*stressx_node)-(stressx_node*stressy_node)+(stressy_node*stressy_node)+(3*stressxy_node*stressxy_node))

        VonMises_node_nn=np.reshape(np.flip(VonMises_node,0),(((ElementsX+1)*(ElementsY+1),1)))
        VonMises_nn=np.reshape(VonMises_element,((ElementsX*ElementsY),1))

        VonMises_node_nn=max(VonMises_node_nn)/VonMises_node_nn
        VonMises_node_nn[VonMises_node_nn>1e4]=0
        VonMises_node_nn=VonMises_node_nn/max(VonMises_node_nn)
        VonMises_nn=max(VonMises_nn)/VonMises_nn
        VonMises_nn[VonMises_nn>1e3]=0
        VonMises_nn=VonMises_nn/max(VonMises_nn)

        
    
    if Stress is False:
        VonMises_nn=[]

    return(MaxDisplacements,StrainEnergy,VonMises_element,VonMises_nn,VonMises_node_nn)


# OPTS

In [ ]:
# -*- coding: utf-8 -*-
"""
Created on Mon Aug 16 10:17:32 2021

@author: nbrow
"""


import argparse 
def parse_opts(args_in = None):
  
    parser=argparse.ArgumentParser()
    ''' Parameters invovled with generic environment building'''
    parser.add_argument('--Main_EX',
                        default=24,
                        type=int,
                        help='Number of X Elements for Larger Environment')
    
    parser.add_argument('--Main_EY',
                        default=24,
                        type=int,
                        help='Number of Y Elements for Larger Environment')
    parser.add_argument('--PR2_EX',
                    default=12,
                    type=int,
                    help='Number of X Elements for Second Environment used in Case of Progressive Refinement')
    parser.add_argument('--PR2_EY',
                    default=12,
                    type=int,
                    help='Number of Y Elements for Second Environment used in Case of Progressive Refinement')
    parser.add_argument('--PR_EX',
                    default=6,
                    type=int,
                    help='Number of X Elements for Smaller Environment used in Case of Progressive Refinement')
    
    parser.add_argument('--PR_EY',
                    default=6,
                    type=int,
                    help='Number of Y Elements for Smaller Environment used in Case of Progressive Refinement')
    
    parser.add_argument('--Lx',
                default=1,
                type=int,
                help='Length of the Structure in the X Direction')
    
    parser.add_argument('--Ly',
                default=1,
                type=int,
                help='Length of the Structure in the Y Direction')
    
    ''' Parameters Invovled with the TopOpt environment'''
    parser.add_argument('--Eta',
                    default=2,
                    type=int,
                    help='Used for dynamic adjusting reward function. Larger eta means lest prevelance'
                        'given towards changes between current and previous reward. Recommend using [2,4]')
    parser.add_argument('--a',
                    default=5,
                    type=int,
                    help='X Coefficient of the Quadratic Reward Sufarce')
    
    parser.add_argument('--b',
                    default=5,
                    type=int,
                    help='Y Coefficient of the Quadratic Reward Sufarce')
    ''' Parameters Involved with the RL Architecture'''
    parser.add_argument('--replace',
                    default=100,
                    type=int,
                    help='Number of iterations between switching the weights from the active network to the target network')
       
    parser.add_argument('--epsilon_dec',
                    default=3.5e-4,
                    type=float,
                    help='Iterative decay amount of the epsilon value used for exploration/explotation')
    parser.add_argument('--eps_end',
                    default=0.01,
                    type=float,
                    help='Smallest Allowable Epsilon value to be used for exploration/explotation')
    
    parser.add_argument('--mem_size',
                    default=30000,
                    type=int,
                    help='Size of the Replay Buffer')
    parser.add_argument('--n_games',
                    default=50_000,
                    type=int,
                    help='Maximum Number of Training Episodes Conducted')
    
    parser.add_argument('--batch_size',
                    default=128,
                    type=int,
                    help='Batch Size that will be taken from the Replay Buffer per training episode')
    
    parser.add_argument('--lr',
                    default=5e-3,
                    type=float,
                    help='Starting Learning Rate for the Network')
    
    parser.add_argument('--gamma',
                    default=0.1,
                    type=float,
                    help='Discount Factor for Future Rewards ')
    parser.add_argument('--Vol_Frac_1',
                    default=0.7,
                    type=float,
                    help='Volume Fraction during first progressive refinement')
    
    parser.add_argument('--Vol_Frac_2',
                    default=0.5,
                    type=float,
                    help='Final Volume Fraction ')
    
    parser.add_argument('--Vol_Frac_3',
                    default=0.25,
                    type=float,
                    help='Final Volume Fraction ')

    parser.add_argument('--SC',
                    default=10,
                    type=float,
                    help='Stress constraint, between 0 and 2 ')
    
    parser.add_argument('--P_Norm',
                    default=10,
                    type=int,
                    help='Smoothing Parameter for P-Norm Global Stress calculation')
    parser.add_argument('--filename_save',
                       default='DDQN_TopOpt_Generalized_CNN_4L_',
                       type=str,
                       help='When training, what name would you like your weights, and figure saved as')
    parser.add_argument('--filename_load',
                       default='DDQN_TopOpt_Generalized_CNN_4L_6by6',
                       type=str,
                       help='When testing, what name is your NN weights saved under')
    parser.add_argument('--Progressive_Refinement',
                       default=True,
                       type=bool)
    parser.add_argument('--LC',
                       default=False,
                       type=bool,
                       help="type in loading conditions manually")
    parser.add_argument('--Load_Checkpoints',
                       default=True,
                       type=bool,
                       help="Load Checkouts. ")
    
    parser.add_argument('--VF_S',
                       default=0,
                       type=int,
                       help="Use vol fraction constraint [0] or stress constraint [1]")
    
    parser.add_argument('--Min_Dist',
                        default=0,
                        type=int,
                        help=
                        "The 0 value serves as a place holder to represent the minimum distance between the bounded" 
                        "and loaded elements in a given load case.")
    parser.add_argument('--Time_Trial',
                       default=True,
                       type=bool,
                       help="Perform Time Trial")
    
    parser.add_argument('--configfile',
                       default='config.json',
                       type=str,
                       help="name of config file. ")
    
    parser.add_argument('--From_App',
                       default=True,
                       type=bool,
                       help="True if being called by an external app. Not sure this is needed. ")

    parser.add_argument('--base_folder',
                       default=r".",
                       type=str,
                       help="Folder where to find saved files. Helpful if not running the app from the main folder.  ")
 

    if args_in:
        # print(f"Using args {args_in}")
        # all_defaults = {}
        # for key in vars(args):
        #     all_defaults[key] = parser.get_default(key)
        args = parser.parse_args("")
    else: args = parser.parse_args()

    return args
    


# NODE_ELEMENT_EXTRACTION

In [ ]:
# -*- coding: utf-8 -*-
"""
Created on Thu Aug 12 11:42:39 2021

@author: nbrow
"""
# from Top_Opt_RL.DQN .FEA_SOLVER_GENERAL import rectangularmesh
import numpy as np
def LC_Nodes(Load_Element,Load_Type,Load_Direction,Lx,Ly,Elements_X,Elements_Y,Counting,Node_Location):
    '''Given the loaded element and loading direction, 
    produce the nodes that should be loaded for the FEA. If the user is testing
     and selects an element not on the exterior edges of the shape
     they will be prompted to select the top/right/bottom/left nodes 
     of the selected element '''
    Go_List,Elem_List,Bottom_List,Top_List,Left_List,Right_List=Element_Lists(Elements_X,Elements_Y)
    Load_Nodes=rectangularmesh(Lx,Ly,Elements_X,Elements_Y)[1][Load_Element,:]
    if Load_Element in Bottom_List:
        Loaded_Node=Load_Nodes[0]
        Loaded_Node2=Load_Nodes[1]
    if Load_Element in Top_List:
        Loaded_Node=Load_Nodes[2]
        Loaded_Node2=Load_Nodes[3]
    if Load_Element in Right_List:
        Loaded_Node=Load_Nodes[1]
        Loaded_Node2=Load_Nodes[2]
    if Load_Element in Left_List:
        Loaded_Node=Load_Nodes[0]
        Loaded_Node2=Load_Nodes[3]
    if Load_Element in Top_List and Load_Element in Right_List:
        if Load_Type==1:
            Loaded_Node=Load_Nodes[1]
            Loaded_Node2=Load_Nodes[2]
        else:
            Loaded_Node=Load_Nodes[2]
            Loaded_Node2=Load_Nodes[3]
    if Load_Element in Top_List and Load_Element in Left_List:
        if Load_Type==1:
            Loaded_Node=Load_Nodes[0]
            Loaded_Node2=Load_Nodes[3]
        else:
            Loaded_Node=Load_Nodes[2]
            Loaded_Node2=Load_Nodes[3]
    if Load_Element in Bottom_List and Load_Element in Right_List:
        if Load_Type==1:
            Loaded_Node=Load_Nodes[1]
            Loaded_Node2=Load_Nodes[2]
        else:
            Loaded_Node=Load_Nodes[0]
            Loaded_Node2=Load_Nodes[1]
    if Load_Element in Bottom_List and Load_Element in Left_List:
        if Load_Type==1:
            Loaded_Node=Load_Nodes[0]
            Loaded_Node2=Load_Nodes[3]
        else:
            Loaded_Node=Load_Nodes[0]
            Loaded_Node2=Load_Nodes[1]
    if Load_Element not in Bottom_List and Load_Element not in Top_List and Load_Element not in Right_List and Load_Element not in Left_List:
        if Load_Type==0 and Load_Direction==-1:
            Loaded_Node=Load_Nodes[0]
            Loaded_Node2=Load_Nodes[1]
        if Load_Type==1 and Load_Direction==1:
            Loaded_Node=Load_Nodes[1]
            Loaded_Node2=Load_Nodes[2]
        if Load_Type==0 and Load_Direction==1:
            Loaded_Node=Load_Nodes[2]
            Loaded_Node2=Load_Nodes[3]
        if Load_Type==1 and Load_Direction==-1:
            Loaded_Node=Load_Nodes[3]
            Loaded_Node2=Load_Nodes[0]
    Loaded_Node=int(Loaded_Node)
    Loaded_Node2=int(Loaded_Node2)   

    return Loaded_Node, Loaded_Node2

def BC_Nodes(Load_Element,Lx,Ly,Elements_X,Elements_Y):

    ''''Given the Boundary Condition Element,produce the 
    corresponding nodes depending on where it's located'''

    _,_,Bottom_List,Top_List,Left_List,Right_List=Element_Lists(Elements_X,Elements_Y)
    Load_Nodes=rectangularmesh(Lx,Ly,Elements_X,Elements_Y)[1][Load_Element,:]
    if Load_Element in Bottom_List:
     
        Loaded_Node=Load_Nodes[0]
        Loaded_Node2=Load_Nodes[1]
    if Load_Element in Top_List:
  
        Loaded_Node=Load_Nodes[2]
        Loaded_Node2=Load_Nodes[3]
    if Load_Element in Right_List:
      
        Loaded_Node=Load_Nodes[1]
        Loaded_Node2=Load_Nodes[2]
    if Load_Element in Left_List:
       
        Loaded_Node=Load_Nodes[0]
        Loaded_Node2=Load_Nodes[3]
    if Load_Element in Top_List and Load_Element in Right_List:
        Loaded_Node=Load_Nodes[2]
        Loaded_Node2=Load_Nodes[2]
    if Load_Element in Top_List and Load_Element in Left_List:
        Loaded_Node=Load_Nodes[3]
        Loaded_Node2=Load_Nodes[3]
    if Load_Element in Bottom_List and Load_Element in Right_List:
        Loaded_Node=Load_Nodes[1]
        Loaded_Node2=Load_Nodes[1]

    if Load_Element in Bottom_List and Load_Element in Left_List:
        Loaded_Node=Load_Nodes[0]
        Loaded_Node2=Load_Nodes[0]
    if Load_Element not in Bottom_List and Load_Element not in Top_List and Load_Element not in Right_List and Load_Element not in Left_List:
        Loaded_Node=Load_Nodes[0]
        Loaded_Node2=Load_Nodes[1]

    Loaded_Node=int(Loaded_Node)
    Loaded_Node2=int(Loaded_Node2)   
    return Loaded_Node, Loaded_Node2

def Element_Lists(Elements_X,Elements_Y):
    '''Simple function that produces a list of all the elements in the matrix '''
    Go_List=[]
    Elem_List=[]
    Go_List=np.append(Go_List,range(0,Elements_X+1))
    Elem_List=np.append(Elem_List,range(0,Elements_X))

    for num in range(0,Elements_Y-1):
        Go_List=np.append(Go_List,(Elements_X+1)*(num+1))
        Go_List=np.append(Go_List,(Elements_X+1)*(num+2)-1)
    Go_List=np.append(Go_List,range((Elements_X*(Elements_Y+1)),(Elements_X*(Elements_Y+2)+1)))
    for num in range(0,Elements_Y-2):
        Elem_List=np.append(Elem_List,(Elements_X*(num+1)))
        Elem_List=np.append(Elem_List,(Elements_X*(num+2)-1))
    Elem_List=np.append(Elem_List,range(Elements_X*(Elements_Y-1),(Elements_X*(Elements_Y))))
    Bottom_List=np.arange(0, Elements_X,1).tolist()
    Top_List=np.arange(Elements_X*(Elements_Y-1),Elements_X*Elements_Y, 1).tolist()
    Left_List=np.arange(0, Elements_X*(Elements_Y),Elements_X).tolist()
    Right_List=np.arange(Elements_X-1,Elements_X*Elements_Y+1,Elements_X).tolist()

    return Go_List, Elem_List, Bottom_List, Top_List,Left_List,Right_List


# MATRIX_TRANSFORMS

In [ ]:
# -*- coding: utf-8 -*-
"""
Created on Thu Aug 12 15:34:50 2021

@author: nbrow
"""
import numpy as np
import math
from scipy.signal import convolve2d
# from Top_Opt_RL.DQN. Node_Element_Extraction import BC_Nodes,LC_Nodes
def action_flip(action,Elements_X,Elements_Y):
    
    '''Given an element that is being loaded, produce the 
    element horizontally, vertically, and diagonally mirrored to it'''
    Action_Mat=np.zeros((1,Elements_X*Elements_Y))
    Action_Mat[0][action]=1
    Action_Mat=np.reshape(Action_Mat,(Elements_X,Elements_Y))
    AM_v=np.reshape(np.flip(Action_Mat,axis=0),(1,Elements_X*Elements_Y))
    AM_h=np.reshape(np.flip(Action_Mat,axis=1),(1,Elements_X*Elements_Y))
    AM_vh=np.reshape(np.flip(np.reshape(AM_v,(Elements_X,Elements_Y)),axis=1),(1,Elements_X*Elements_Y))
    action_v=np.where(AM_v[0]==1)[0][0]
    action_h=np.where(AM_h[0]==1)[0][0]
    action_vh=np.where(AM_vh[0]==1)[0][0]
    return action_v,action_h,action_vh 

def obs_flip(observation,Elements_X,Elements_Y):
    '''Given an observation, produce the observations 
    that are horizontally, vertically, and diagonally mirrored'''
    
    
    observation_v=np.zeros((Elements_X,Elements_Y,3))
    observation_h=np.zeros((Elements_X,Elements_Y,3))
    observation_vh=np.zeros((Elements_X,Elements_Y,3))
    for Flip in range(0,3):
        observation_v[:,:,Flip]=np.flip(observation[:,:,Flip],axis=0)
        observation_h[:,:,Flip]=np.flip(observation[:,:,Flip],axis=1)
    for Flip in range(0,3):
        observation_vh[:,:,Flip]=np.flip(observation_v[:,:,Flip],axis=1)
        
    return observation_v,observation_h,observation_vh

def Mesh_Triming(env,Elements_X,Elements_Y):
    '''This function can be used to eleminate elements that are only 
    singularly connected to the rest of the matrix and do not provide 
    substantial support to the rest of the structure. It can be thought
    of as a shaving algorithm to help catch single elements the RL agent mises 
    at the end'''
    Final=False
    Count_1=list(env.VoidCheck).count(0)
    while not Final:
        VC_Hold=np.zeros((Elements_X+2,Elements_Y+2))
        VC_Hold[1:Elements_X+1,1:Elements_Y+1]=np.reshape(env.VoidCheck,(Elements_X,Elements_Y))
        c = convolve2d(VC_Hold, np.array([[0,1,0],[1,0,1],[0,1,0]]), mode='valid')
        VV=VC_Hold[1:Elements_X+1,1:Elements_Y+1]
        VV_Loc=np.where(np.reshape(VV,(1,(Elements_X*Elements_Y)))==0)[1]
        c=np.reshape(c,(1,Elements_X*Elements_Y))[0]
        c[VV_Loc]=0
        c_Loc=np.where(c==1)[0]
        for i in range(0,len(env.BC)):
            c_Loc=np.delete(c_Loc,np.where(c_Loc==env.BC[i]))
        if len(c_Loc)>0:
            Final=False
        else:
            Final=True

        if len(c_Loc)>0:
            try:
                env.VoidCheck[c_Loc]=0 
            except TypeError:
                env.VoidCheck[c_Loc[0]]=0
    Count_2=list(env.VoidCheck).count(0)
    return Count_2-Count_1  

def Condition_Transform(Lx,Ly,Old_EX,Old_EY,New_EX,New_EY,BC_Elements,LC_Elements,Load_Type,Load_Direction):
    New_BC_Elements=[]
    New_BC_Nodes=[]
    New_LC_Elements=[]
    New_LC_Nodes=[]
    for BC in range(0,len(BC_Elements)):
        BC1_E=BC_Elements[BC]
        Row_BC1_E=math.floor(BC1_E/New_EY)
        Col_BC1_E=math.floor(round(math.modf(BC1_E/New_EX)[0],2)*New_EX)

        Old_X_Perc_BC1_E=Row_BC1_E/New_EY
        Old_Y_Perc_BC1_E=Col_BC1_E/New_EX

        New_Row_BC1_E=math.floor((Old_X_Perc_BC1_E*Old_EX)+0.001)
        New_Col_BC1_E=math.floor((Old_Y_Perc_BC1_E*Old_EY)+0.001)
        New_BC1_E=(New_Row_BC1_E*Old_EX)+New_Col_BC1_E
        New_BC1,New_BC2=BC_Nodes(New_BC1_E,Lx,Ly,Old_EX,Old_EY)
        New_BC_Elements=np.append(New_BC_Elements,New_BC1_E)
        New_BC_Nodes=np.append(New_BC_Nodes,New_BC1)
        New_BC_Nodes=np.append(New_BC_Nodes,New_BC2)
    
    for LC_ in range(0,len(LC_Elements)):
        LC=LC_Elements[LC_]
        Row_LC=math.floor(LC/New_EY)
        Col_LC=math.floor(round(math.modf(LC/New_EX)[0],2)*New_EX)
        Old_X_Perc=Row_LC/New_EY
        Old_Y_Perc=Col_LC/New_EX
        New_Row_LC=math.floor((Old_X_Perc*Old_EX)+0.001)
        New_Col_LC=math.floor((Old_Y_Perc*Old_EY)+0.001)
        New_LC_E=(New_Row_LC*Old_EX)+New_Col_LC
        New_LC1,New_LC2=LC_Nodes(New_LC_E,Load_Type[LC_],Load_Direction[LC_],Lx,Ly,Old_EX,Old_EY,LC_,Node_Location=False)
        New_LC_Elements=np.append(New_LC_Elements,New_LC_E)
        New_LC_Nodes=np.append(New_LC_Nodes,New_LC1)
        New_LC_Nodes=np.append(New_LC_Nodes,New_LC2)
    
    return New_BC_Nodes,New_BC_Elements,New_LC_Elements,New_LC_Nodes,Load_Direction

def Mesh_Transform(Old_EX,Old_EY,New_EX,New_EY,Config):

    Config=np.reshape(Config,(Old_EX,Old_EY))
    Old_X_Perc=100/Old_EX
    Old_Y_Perc=100/Old_EY
    New_X_Perc=100/New_EX
    New_Y_Perc=100/New_EY
    
    New_Config=np.zeros((New_EY,New_EX))
    for i in range(Old_EX):
        for j in range(Old_EY):
            if Config[i,j]==1:
                Old_X_Min=j*Old_X_Perc
                Old_X_Max=(j+1)*Old_X_Perc
                Old_Y_Min=i*Old_Y_Perc
                Old_Y_Max=(i+1)*Old_Y_Perc
                New_X_Block_Max=math.ceil(Old_X_Max/New_X_Perc)
                if New_X_Block_Max!=0:
                    New_X_Block_Max-=1
                New_X_Block_Min=math.floor(Old_X_Min/New_X_Perc)
                #if New_X_Block_Min!=0:
                #    New_X_Block_Min-=1
                New_Y_Block_Max=math.ceil(Old_Y_Max/New_Y_Perc)
                if New_Y_Block_Max!=0:
                    New_Y_Block_Max-=1
                New_Y_Block_Min=math.floor(Old_Y_Min/New_Y_Perc)
                #if New_Y_Block_Min!=0:
                #    New_Y_Block_Min-=1
                New_Config[New_Y_Block_Min:New_Y_Block_Max+1,New_X_Block_Min:New_X_Block_Max+1:1]=1
    New_Config=np.reshape(New_Config,(1,New_EX*New_EY))
    
    return list(New_Config[0])


# TOPOPT_ENV_FUNCTIONS

In [ ]:
# -*- coding: utf-8 -*-
"""
Created on Thu Aug 12 12:00:02 2021

@author: nbrow
"""
from gym import Env
from gym.spaces import Discrete
import numpy as np
import itertools
# from Top_Opt_RL.DQN.Node_Element_Extraction import BC_Nodes,LC_Nodes,Element_Lists
# from Top_Opt_RL.DQN.FEA_SOLVER_GENERAL import FEASolve, isolate_largest_group_original, rectangularmesh
import math
import copy
# from Top_Opt_RL.DQN. Matrix_Transforms import Condition_Transform
import random
import sys
import os
 
class TopOpt_Gen(Env):
    def __init__(self,Elements_X,Elements_Y,Vol_Frac,SC,opts):
        #Actons we can take... remove any of the blocks
        self.EX=Elements_X
        self.p=opts.P_Norm
        self.RS=Reward_Surface(opts)[0]
        self.RV=Reward_Surface(opts)[1]
        self.SC=SC
        self.Lx=opts.Lx
        self.EY=Elements_Y
        self.Ly=opts.Ly
        self.action_space=Discrete(self.EX*self.EY)
        self.eta=opts.Eta
        self.Vol_Frac=Vol_Frac
    def step(self,action,observation,Last_Reward,load_checkpoint,env,PR,FEA_Skip):
        #Apply Action
        self.Counter+=1    
        # evaluate it on grid

        rs_place=self.VoidCheck[int(action)]
        self.VoidCheck[int(action)]=0
        ElementMat=np.reshape(self.VoidCheck,(self.EX,self.EY))
        SingleCheck=isolate_largest_group_original(ElementMat)
        It=list(self.VoidCheck).count(0)
        if rs_place==1 and action not in self.BC and SingleCheck[1]==True:
            done=False
            if It>=math.ceil((self.EX*self.EY)*(1-self.Vol_Frac)) and load_checkpoint or It>=math.ceil((self.EX*self.EY)*(1-self.Vol_Frac)) and PR:
                done=True
            if self.Counter==1 or (self.Counter/FEA_Skip)==int(self.Counter/FEA_Skip):
                Run_Results=FEASolve(list(self.VoidCheck),self.Lx,self.Ly,self.EX,self.EY,self.LC_Nodes,self.Load_Directions,self.BC_Nodes,Stress=True)
                self.Max_SE_Ep=np.max(Run_Results[1])
                if (env.P_Norm/(sum(sum([number**self.p for number in np.reshape(Run_Results[2],(1,self.EX*self.EY))]))**(1/self.p)))<(1-float(self.SC)):

                    done=True
                    print('STRESS CONSTRAINT HIT!')
            else:
                self.Stress_state=np.reshape(self.Stress_state,(1,self.EX*self.EY))
                self.Stress_state[0][action]=0
                self.Stress_state=np.reshape(self.Stress_state,(self.EX,self.EY))
            
            if abs(self.Max_SE_Tot/self.Max_SE_Ep)>=1 or abs(It/(self.EX*self.EY))>=1 or self.Max_SE_Tot==0 or self.Max_SE_Ep==0:
                reward=-1
                done=True
            else:
                reward = self.RS[(int((self.Max_SE_Tot/self.Max_SE_Ep)*1000))-1,int((It/(self.EX*self.EY))*1000)-1]
                if not load_checkpoint:
                    reward2=self.RV[int(1-(np.reshape(self.Stress_state,(self.EX*self.EY,1))[action])*1000)-1]
                    reward=np.mean([reward,reward2])
            if self.Counter==1 or (self.Counter/FEA_Skip)==int(self.Counter/FEA_Skip):
             
                self.Stress_state=Run_Results[3]
                self.Stress_state=np.reshape(self.Stress_state,(self.EX,self.EY))
            self.state=np.zeros((self.EX,self.EY,3))
            self.state[:,:,0]=self.Stress_state
            self.state[:,:,1]=self.BC_state
            self.state[:,:,2]=self.LC_state
        else:
            """If the removed block has already been removed, leads to a non-singular
            body or one of the Boundary condition blocks, the agent should be severely punished (-1)"""
            Run_Results=FEASolve(list(self.VoidCheck),self.Lx,self.Ly,self.EX,self.EY,self.LC_Nodes,self.Load_Directions,self.BC_Nodes,Stress=True)
            self.Max_SE_Ep=np.max(Run_Results[1])
            self.Stress_state=Run_Results[3]
            self.Stress_state=np.reshape(self.Stress_state,(self.EX,self.EY))
            self.state=np.zeros((self.EX,self.EY,3))
            self.state[:,:,0]=self.Stress_state
            self.state[:,:,1]=self.BC_state
            self.state[:,:,2]=self.LC_state
            reward=-1
            done=True
            if rs_place==1:
                self.VoidCheck[int(action)]=1
            
        reward+=1e-4
        Last_Reward+=1e-4
        rho=((reward)-(Last_Reward))/min([reward,Last_Reward])
        if reward>Last_Reward:
            llambda=1
        else:
            llambda=-1
        x=rho+llambda
        f_x=math.atan(x*(math.pi/2)*(1/self.eta))
        reward=reward+(f_x-llambda)*abs(reward)
        

        return self.state, reward, done, It
    
    def render(self,mode='human'):
        RenderMat=copy.deepcopy(self.VoidCheck)
        for RM in range(0,len(self.BC_Elements)):
            RenderMat[int(self.BC_Elements[RM])]=2
            RenderMat[int(self.BC_Elements[RM])]=2
        for RM in range(0,len(self.LC_Elements)):
            RenderMat[int(self.LC_Elements[RM])]=4
        RenderMat=np.reshape(RenderMat,(self.EX,self.EY))
        print(np.flip(RenderMat,0))
        print('')
        return 
        
    def reset(self):

        self.Results=FEASolve(self.VoidCheck,self.Lx,self.Ly,self.EX,self.EY,self.LC_Nodes,self.Load_Directions,self.BC_Nodes,Stress=True)
        self.Stress_state=self.Results[3]
        self.P_Norm=sum(sum([number**self.p for number in np.reshape(self.Results[2],(1,self.EX*self.EY))]))**(1/self.p)        #self.Stress_state=list(np.array(self.Stress_state)
        self.Stress_state=np.reshape(self.Stress_state,(self.EX,self.EY))
        self.state=np.zeros((self.EX,self.EY,3))
        self.state[:,:,0]=self.Stress_state
        self.state[:,:,1]=self.BC_state
        self.state[:,:,2]=self.LC_state
        self.Counter=0

        return self.state
    def reset_conditions(self):
        self.Max_SE_Tot=0
        self.VoidCheck=np.ones((1,self.EX*self.EY))
        self.VoidCheck=list(self.VoidCheck[0])
        self.VoidCheck=np.array(self.VoidCheck)

        while self.Max_SE_Tot<=0 or self.Max_SE_Tot>5000:        
            self.BC_Elements=[]
            self.BC_Nodes=[]
            self.LC_Elements=[]
            self.LC_Nodes=[]
            self.BC=[]
            self.Load_Types=[]
            self.Load_Directions=[]
            self.BC_Elements=np.append(self.BC_Elements,int(random.choice([i for i in Element_Lists(self.EX,self.EY)[1]])))
            self.BC_Elements=np.append(self.BC_Elements,int(random.choice([i for i in Element_Lists(self.EX,self.EY)[1]])))
            while self.BC_Elements[0]==self.BC_Elements[1]:
                self.BC_Elements[1]=int(random.choice([i for i in Element_Lists(self.EX,self.EY)[1]]))
            self.BC_Nodes=np.append(self.BC_Nodes,BC_Nodes(int(self.BC_Elements[0]),self.Lx,self.Ly,self.EX,self.EY)[0])
            self.BC_Nodes=np.append(self.BC_Nodes,BC_Nodes(int(self.BC_Elements[0]),self.Lx,self.Ly,self.EX,self.EY)[1])
            self.BC_Nodes=np.append(self.BC_Nodes,BC_Nodes(int(self.BC_Elements[1]),self.Lx,self.Ly,self.EX,self.EY)[0])
            self.BC_Nodes=np.append(self.BC_Nodes,BC_Nodes(int(self.BC_Elements[1]),self.Lx,self.Ly,self.EX,self.EY)[1])

      
            self.LC_Elements=np.append(self.LC_Elements,int(random.choice([i for i in Element_Lists(self.EX,self.EY)[1]])))
            while self.LC_Elements[0] in self.BC_Elements:
                self.LC_Elements[0]=int(random.choice([i for i in Element_Lists(self.EX,self.EY)[1]]))
            
            self.BC_set=np.append(self.BC_Elements,self.LC_Elements)
            self.LC_state=list(np.zeros((1,self.EX*self.EY))[0])
            for LCS in range(0,len(self.LC_Elements)):
                self.LC_state[int(self.LC_Elements[LCS])]=1
            self.LC_state=np.reshape(self.LC_state,(self.EX,self.EY))
            self.Load_Types=np.append(self.Load_Types,random.choice([0,1]))
            self.LC_Nodes=np.append(self.LC_Nodes,LC_Nodes(int(self.LC_Elements[0]),self.Load_Types,self.Load_Directions,self.Lx,self.Ly,self.EX,self.EY,LCS,Node_Location=False)[0])
            self.LC_Nodes=np.append(self.LC_Nodes,LC_Nodes(int(self.LC_Elements[0]),self.Load_Types,self.Load_Directions,self.Lx,self.Ly,self.EX,self.EY,LCS,Node_Location=False)[1])
            if self.Load_Types[0]==0: #Load will be applied vertically
                self.LC_Nodes[0]+=((self.EX+1)*(self.EY+1))
                self.LC_Nodes[1]+=((self.EX+1)*(self.EY+1))
            self.Load_Directions=np.append(self.Load_Directions,random.choice([-1,1])) #1 for Compressive Load, -1 for tensile load
            self.BC=np.append(self.BC,self.BC_Elements)
            self.BC=np.append(self.BC,self.LC_Elements)
            self.BC_state=list(np.zeros((1,self.EX*self.EY))[0])
            for BCS in range(0,len(self.BC_Elements)):
                self.BC_state[int(self.BC_Elements[BCS])]=1
            self.BC_state=np.reshape(self.BC_state,(self.EX,self.EY))
            self.Results=FEASolve(self.VoidCheck,self.Lx,self.Ly,self.EX,self.EY,self.LC_Nodes,self.Load_Directions,self.BC_Nodes,Stress=True)
            self.Max_SE_Tot=self.Results[1]
            
    def primer_cond(self,EX,EY):
        self.BC=[]
        self.BC=np.append(self.BC,self.BC_Elements)
        self.BC=np.append(self.BC,self.LC_Elements)
        self.BC_state=list(np.zeros((1,EX*EY))[0])
        for BCS in range(0,len(self.BC_Elements)):
            self.BC_state[int(self.BC_Elements[BCS])]=1
            self.BC_state=np.reshape(self.BC_state,(EX,EY))
            self.LC_state=list(np.zeros((1,EX*EY))[0])
            for LCS in range(0,len(self.LC_Elements)):
                self.LC_state[int(self.LC_Elements[LCS])]=1
                self.LC_state=np.reshape(self.LC_state,(EX,EY))
                self.Results=FEASolve(self.VoidCheck,self.Lx,self.Ly,self.EX,self.EY,self.LC_Nodes,self.Load_Directions,self.BC_Nodes,Stress=True)
                self.Max_SE_Tot=np.max(self.Results[1])

def Prog_Refine_Act(agent_primer,env,env_primer,load_checkpoint,Testing,opts,Small_EX,Small_EY,Time_Trial,From_App,FEA_Skip):
    '''This function will deliver the optimal topology of the smaller sized environment.
    This final topology will then be transformed into the equivalent topology at the 
    larger selected size. This larger topology will then be based back to the main function
    and the topology removal process will continue.'''
    Stable=False
    while not Stable:
        env_primer.BC_Nodes,env_primer.BC_Elements,env_primer.LC_Elements,env_primer.LC_Nodes,env_primer.Load_Directions=Condition_Transform(opts.Lx,opts.Ly,Small_EX,Small_EY,opts.Main_EX,opts.Main_EY,env.BC_Elements,env.LC_Elements,env.Load_Types,env.Load_Directions)
        
        LN_Hold=env_primer.LC_Nodes
        LT_Count=0
        for Counting in range(0,len(env_primer.LC_Nodes)):
            if Counting/2==int(Counting/2) and Counting/2!=0:
                LT_Count+=1
            if env.Load_Types[LT_Count]==0:
                env_primer.LC_Nodes[Counting]+=((Small_EX+1)*(Small_EY+1))
        env_primer.primer_cond(Small_EX,Small_EY)
        for Check in range(0,len(LN_Hold)):
            if LN_Hold[Check] in env_primer.BC_Nodes:
                if load_checkpoint:
                    print('-------------------------------')
                    print('Illegal Combination of BC and LC')
                    print('-------------------------------')
                    sys.exit()
                env.reset_conditions()
            else:
                Stable=True
    primer_done=False
    observation_primer=env_primer.reset()
    Last_Reward=0
    while not primer_done:
        Testing=True
        action = agent_primer.choose_action(observation_primer,load_checkpoint,Testing)
        observation_primer_, reward, primer_done, It = env_primer.step(action,observation_primer,Last_Reward,load_checkpoint,env,FEA_Skip=FEA_Skip,PR=True)
        observation_primer = observation_primer_
        Last_Reward=reward
        if load_checkpoint and not Time_Trial:
            env_primer.render()
    Last_Reward=0
    if Testing and not From_App:
        env_primer.render()
def Min_Dist_Calc(UC_,opts):
    
    BC=[int(x)-1 for x in UC_['bcs']]
    Right=[int(x)-1 for x in UC_['rights']]
    Left=[int(x)-1 for x in UC_['lefts']]
    Up=[int(x)-1 for x in UC_['ups']]
    Down=[int(x)-1 for x in UC_['downs']]
    Elements=[]
    Elements=np.append(Elements,Right)
    Elements=np.append(Elements,Left)
    Elements=np.append(Elements,Up)
    Elements=np.append(Elements,Down)
    Elements=np.append(Elements,BC)

    Len_Mat=[]
    for i in range(0,len(Elements)):
        for j in range(i+1,len(Elements)):
            Row_E1=math.floor(Elements[i]/opts.Main_EX)
            Col_E1=math.ceil(math.modf(Elements[i]/opts.Main_EX)[0]*opts.Main_EX)
            Row_E2=math.floor(Elements[j]/opts.Main_EX)
            Col_E2=math.ceil(math.modf(Elements[j]/opts.Main_EX)[0]*opts.Main_EX)
            E2_E1=abs(Row_E2-Row_E1)+abs(Col_E2-Col_E1)
            Len_Mat=np.append(Len_Mat,E2_E1)
    Len_Mat=list(Len_Mat)
    while len(Len_Mat)>len(Elements):
        Len_Mat.remove(max(Len_Mat))
    return sum(Len_Mat)
def App_Inputs(env,env_primer,env_primer2,opts,User_Conditions):
    '''To improve the adaptability of this method, a web-app has been developed
    using Heroku The web-app will provide an interactive environment for the user
    to select the boundary and loading conditions. The BCs and LCs will be imported as 
    a .json file and distributed accordingly similar to the User_Input function'''
    env.BC_Elements=[]
    env.LC_Elements=[]
    env.BC_Nodes=[]
    env.LC_Nodes=[]
    env.Load_Types=[]
    env.Load_Directions=[]

    BC=[int(x)-1 for x in User_Conditions['bcs']]
    Right=[int(x)-1 for x in User_Conditions['rights']]
    Left=[int(x)-1 for x in User_Conditions['lefts']]
    Up=[int(x)-1 for x in User_Conditions['ups']]
    Down=[int(x)-1 for x in User_Conditions['downs']]
    env.BC_Elements=np.append(env.BC_Elements,BC)
    env.Vol_Frac=User_Conditions['volfraction']
    Min_Dist=Min_Dist_Calc(User_Conditions,opts)
    env_primer.Vol_Frac=env.Vol_Frac+3*(Min_Dist/(opts.Main_EX*opts.Main_EY))
    if env_primer.Vol_Frac>0.7:
        env_primer.Vol_Frac=0.7
    env_primer2.Vol_Frac=env.Vol_Frac+2*(Min_Dist/(opts.Main_EX*opts.Main_EY))
    if env_primer2.Vol_Frac>0.5:
        env_primer2.Vol_Frac=0.5
    env.LC_Elements=np.append(env.LC_Elements,Right).astype('int')
    env.Load_Types=np.append(env.Load_Types,[1]*len(Right)).astype('int')
    env.Load_Directions=np.append(env.Load_Directions,[-1]*len(Right)).astype('int')
    
    env.LC_Elements=np.append(env.LC_Elements,Left).astype('int')
    env.Load_Types=np.append(env.Load_Types,[1]*len(Left)).astype('int')
    env.Load_Directions=np.append(env.Load_Directions,[-1]*len(Left)).astype('int')
    
    env.LC_Elements=np.append(env.LC_Elements,Up).astype('int')
    env.Load_Types=np.append(env.Load_Types,[0]*len(Up)).astype('int')
    env.Load_Directions=np.append(env.Load_Directions,[-1]*len(Up)).astype('int')
    
    env.LC_Elements=np.append(env.LC_Elements,Down).astype('int')
    env.Load_Types=np.append(env.Load_Types,[0]*len(Down)).astype('int')
    env.Load_Directions=np.append(env.Load_Directions,[-1]*len(Down)).astype('int')
    for Counting in range(0,len(env.LC_Elements)):
        if env.Load_Types[Counting]==0:
            LC_New_Nodes=LC_Nodes(int(env.LC_Elements[Counting]),env.Load_Types[Counting],env.Load_Directions[Counting],env.Lx,env.Ly,env.EX,env.EY,Counting,Node_Location=True)
            env.LC_Nodes=np.append(env.LC_Nodes,LC_New_Nodes[0]+(opts.Main_EX+1)*(opts.Main_EY+1))
            env.LC_Nodes=np.append(env.LC_Nodes,LC_New_Nodes[1]+(opts.Main_EX+1)*(opts.Main_EY+1))
        else:
            LC_New_Nodes=LC_Nodes(int(env.LC_Elements[Counting]),env.Load_Types[Counting],env.Load_Directions[Counting],env.Lx,env.Ly,env.EX,env.EY,Counting,Node_Location=True)
            env.LC_Nodes=np.append(env.LC_Nodes,LC_New_Nodes[0])
            env.LC_Nodes=np.append(env.LC_Nodes,LC_New_Nodes[1])
    for Counting in range(0,len(env.BC_Elements)):
        env.BC_Nodes=np.append(env.BC_Nodes,BC_Nodes(int(env.BC_Elements[Counting]),env.Lx,env.Ly,env.EX,env.EY)[0])
        env.BC_Nodes=np.append(env.BC_Nodes,BC_Nodes(int(env.BC_Elements[Counting]),env.Lx,env.Ly,env.EX,env.EY)[1])

    env.LC_state=list(np.zeros((1,(opts.Main_EX)*(opts.Main_EY)))[0])
    for LCS in range(0,len(env.LC_Elements)):
                env.LC_state[int(env.LC_Elements[LCS])]=1
    env.LC_state=np.reshape(env.LC_state,(opts.Main_EX,opts.Main_EY))
    env.BC=[]
    env.BC=np.append(env.BC,env.BC_Elements)
    env.BC=np.append(env.BC,env.LC_Elements)
    env.BC_state=list(np.zeros((1,(opts.Main_EX)*(opts.Main_EY)))[0])
    for BCS in range(0,len(env.BC_Elements)):
        env.BC_state[int(env.BC_Elements[BCS])]=1
    env.BC_state=np.reshape(env.BC_state,(opts.Main_EX,opts.Main_EY))
    env.Max_SE_Tot=np.max((FEASolve(env.VoidCheck,opts.Lx,opts.Ly,opts.Main_EX,opts.Main_EY,env.LC_Nodes,env.Load_Directions,env.BC_Nodes,Stress=True)[1]))
    
    
def User_Inputs(env,opts):
    '''When testing a trained agent, the user will be prompted to select
    a single element to act as the loaded element, and two elements to act as the boundary 
    condition elements. Depending on where the elements are located, the nodes
    corresponding to these elements will be selected'''
    print(np.flip(np.reshape(range(0,(opts.Main_EX)*(opts.Main_EY)),(opts.Main_EX,opts.Main_EY)),0))
    BC_Count=int(input('How many boundary elements would you like to have: '))
    env.BC_Elements=[]
    env.LC_Elements=[]
    env.BC_Nodes=[]
    env.LC_Nodes=[]
    env.Load_Types=[]
    env.Load_Directions=[]
    for Counting in range(0,BC_Count):
        env.BC_Elements=np.append(env.BC_Elements,int(input('Please select an element to apply Boundary condition #'+str(Counting+1)+': ')))
        if env.BC_Elements[Counting]>(opts.Main_EX)*(opts.Main_EY) or env.BC_Elements[Counting]<0 or env.BC_Elements[Counting]!=int(env.BC_Elements[Counting]):
            print('Code Terminated By User...')
            sys.exit()
    print(np.flip(np.reshape(range(0,(opts.Main_EX*opts.Main_EY)),(opts.Main_EX,opts.Main_EY)),0))
    LC_Count=int(input('How many loading elements would you like to have: '))
    

    for Counting in range(0,LC_Count):
        env.LC_Elements=np.append(env.LC_Elements,int(input('Please select an element to apply the load #'+str(Counting+1)+': ')))
        if env.LC_Elements[Counting]>(opts.Main_EX)*(opts.Main_EY) or env.LC_Elements[Counting]<0 or env.LC_Elements[Counting]!=int(env.LC_Elements[Counting]):
            print('Code Terminated By User...')
            sys.exit()
        env.Load_Types=np.append(env.Load_Types,int(input('Input 0 for a Vertical Load or Input 1 for a Horizontal load for this element: ')))
        env.Load_Directions=np.append(env.Load_Directions,int(input('Input -1 for a tensile load or Input 1 for a compressive load for this element: ')))
    for Counting in range(0,LC_Count):
        if env.Load_Types[Counting]==0:
            LC_New_Nodes=LC_Nodes(int(env.LC_Elements[Counting]),env.Load_Types[Counting],env.Load_Directions[Counting],env.Lx,env.Ly,env.EX,env.EY,Counting,Node_Location=True)
            env.LC_Nodes=np.append(env.LC_Nodes,LC_New_Nodes[0]+(opts.Main_EX+1)*(opts.Main_EY+1))
            env.LC_Nodes=np.append(env.LC_Nodes,LC_New_Nodes[1]+(opts.Main_EX+1)*(opts.Main_EY+1))
        else:
            LC_New_Nodes=LC_Nodes(int(env.LC_Elements[Counting]),env.Load_Types[Counting],env.Load_Directions[Counting],env.Lx,env.Ly,env.EX,env.EY,Counting,Node_Location=True)
            env.LC_Nodes=np.append(env.LC_Nodes,LC_New_Nodes[0])
            env.LC_Nodes=np.append(env.LC_Nodes,LC_New_Nodes[1])
    for Counting in range(0,BC_Count):
        env.BC_Nodes=np.append(env.BC_Nodes,BC_Nodes(int(env.BC_Elements[Counting]),env.Lx,env.Ly,env.EX,env.EY)[0])
        env.BC_Nodes=np.append(env.BC_Nodes,BC_Nodes(int(env.BC_Elements[Counting]),env.Lx,env.Ly,env.EX,env.EY)[1])

    env.LC_state=list(np.zeros((1,(opts.Main_EX)*(opts.Main_EY)))[0])
    for LCS in range(0,len(env.LC_Elements)):
                env.LC_state[int(env.LC_Elements[LCS])]=1
    env.LC_state=np.reshape(env.LC_state,(opts.Main_EX,opts.Main_EY))
    env.BC=[]
    env.BC=np.append(env.BC,env.BC_Elements)
    env.BC=np.append(env.BC,env.LC_Elements)
    env.BC_state=list(np.zeros((1,(opts.Main_EX)*(opts.Main_EY)))[0])
    for BCS in range(0,len(env.BC_Elements)):
        env.BC_state[int(env.BC_Elements[BCS])]=1
    env.BC_state=np.reshape(env.BC_state,(opts.Main_EX,opts.Main_EY))
    env.Max_SE_Tot=np.max((FEASolve(env.VoidCheck,opts.Lx,opts.Ly,opts.Main_EX,opts.Main_EY,env.LC_Nodes,env.Load_Directions,env.BC_Nodes,Stress=True)[1]))
    
    return 
def Testing_Inputs(env,opts):
    '''Every 200 episodes, the boundary and loading conditions
    should be set as those of a cantilever beam to monitor the progress
    of the agents learning'''
    env.BC_Nodes=np.array([0,0,opts.Main_EX*opts.Main_EY,opts.Main_EX*opts.Main_EY])
    env.LC_Nodes=np.array([opts.Main_EX+(opts.Main_EX+1)*(opts.Main_EY+1),opts.Main_EX-1+(opts.Main_EX)*(opts.Main_EY+1)])

    env.LC_Elements=np.array([np.where(rectangularmesh(opts.Lx,opts.Ly,opts.Main_EX,opts.Main_EY)[1]==env.LC_Nodes[0]-((opts.Main_EX+1)*(opts.Main_EY+1)))[0][0]])
    env.BC_Elements=[0,(opts.Main_EX)*(opts.Main_EY-1)]
    env.LC_state=list(np.zeros((1,(opts.Main_EX)*(opts.Main_EY)))[0])
    env.LC_state[env.LC_Elements[0]]=1
    env.LC_state=np.reshape(env.LC_state,(opts.Main_EX,opts.Main_EY))
    env.Load_Types=[0]
    env.Load_Directions=[-1] #1 for Compressive Load, -1 for tensile load
    env.BC=[]
    env.BC=np.append(env.BC,env.BC_Elements)
    env.BC=np.append(env.BC,env.LC_Elements)
    env.BC_state=list(np.zeros((1,(opts.Main_EX)*(opts.Main_EY)))[0])
    env.BC_state[env.BC_Elements[0]]=1
    env.BC_state[env.BC_Elements[1]]=1
    env.BC_state=np.reshape(env.BC_state,(opts.Main_EX,opts.Main_EY))
    env.Max_SE_Tot=np.max((FEASolve(env.VoidCheck,opts.Lx,opts.Ly,opts.Main_EX,opts.Main_EY,env.LC_Nodes,env.Load_Directions,env.BC_Nodes,Stress=True)[1]))

def Testing_Info(env,env_primer,env_primer2,opts,score,Progressive_Refinement,From_App,Fixed):
    '''Function that outputs the results of a testing trial. The results include
    the score based on the reward function, the final strain energy, and if needed
    the number of arbitrary blocks removed by the shaving algorithm'''
    if not From_App:
        import matplotlib.pyplot as plt
        print('----------------')
    
        print('The final topology: ')
        for BC_Count in range(0,len(env.BC_Elements)):
            print('BC Element #'+str(BC_Count)+': '+str(int(env.BC_Elements[BC_Count])))
        for LC_Count in range(0,len(env.LC_Elements)):
            print('LC Element #'+str(LC_Count)+': '+str(int(env.LC_Elements[LC_Count])))
            if env.Load_Types[LC_Count]==0:
                Load_Types='Vertical'
            else:
                Load_Types='Horizontal'
            if env.Load_Directions[LC_Count]==-1:
                Load_Dir='Tensile'
            else:
                Load_Dir='Compressive'
            print('Load Type: '+Load_Dir)
            print('Load Direction: '+Load_Types)
      
        if Progressive_Refinement:
            env_primer.render()
            env_primer2.render()
        env.render()
        Final_Results=FEASolve(list(env.VoidCheck),opts.Lx,opts.Ly,opts.Main_EX,opts.Main_EY,env.LC_Nodes,env.Load_Directions,env.BC_Nodes,Stress=True)
        print('Strain Energy for Final Topology: '+str(round(np.max(Final_Results[1]),1)))
        p=opts.P_Norm
        print('Maximum P_Norm Stress Perc Increase: '+str(round(1-(env.P_Norm/sum(sum([number**p for number in np.reshape(Final_Results[2],(1,env.EX*env.EY))]))**(1/p)),2)))
        print('Final Volume Fraction: '+str(round(1-(list(env.VoidCheck).count(0)/(env.EX*env.EY)),3)))
        
        print('----------------')
        Mat_Plot=copy.deepcopy(env_primer.VoidCheck)
        plt.figure(1)
        for BC_Count in range(0,len(env_primer.BC_Elements)):
            Mat_Plot[int(env_primer.BC_Elements[BC_Count])]=3
        for LC_Count in range(0,len(env_primer.LC_Elements)):
            Mat_Plot[int(env_primer.LC_Elements[LC_Count])]=2
        plt.subplot(221)
        plt.imshow(np.flip(np.reshape(Mat_Plot,(opts.PR_EX,opts.PR_EY)),axis=0),cmap='Blues')
        Mat_Plot=copy.deepcopy(env_primer2.VoidCheck)
        for BC_Count in range(0,len(env_primer2.BC_Elements)):
            Mat_Plot[int(env_primer2.BC_Elements[BC_Count])]=3
        for LC_Count in range(0,len(env_primer2.LC_Elements)):
            Mat_Plot[int(env_primer2.LC_Elements[LC_Count])]=2
            plt.subplot(222)
        plt.imshow(np.flip(np.reshape(Mat_Plot,(opts.PR2_EX,opts.PR2_EY)),axis=0),cmap='Blues')
        Mat_Plot=copy.deepcopy(env.VoidCheck)
        for BC_Count in range(0,len(env.BC_Elements)):
            Mat_Plot[int(env.BC_Elements[BC_Count])]=3
        for LC_Count in range(0,len(env.LC_Elements)):
            Mat_Plot[int(env.LC_Elements[LC_Count])]=2
        plt.subplot(224)
        plt.imshow(np.flip(np.reshape(Mat_Plot,(opts.Main_EX,opts.Main_EY)),axis=0),cmap='Blues')
        plt.show()
        App_Plot={}
    else:
        Mat_Plot=copy.deepcopy(env.VoidCheck)
        App_Plot={}
        App_Plot['Topology']=[]
        App_Plot['SE']=[]
        App_Plot['VF']=[]
        App_Plot['Topology'].append([str(x) for x in Mat_Plot])
        App_Plot['SE'].append(str(round(env.Max_SE_Ep,1)))
        App_Plot['VF'].append(str(round(1-(list(env.VoidCheck).count(0)/(env.EX*env.EY)),3)))
    return App_Plot
        
def poly_matrix(x, y, order=2):
    """ Function to produce a matrix built on a quadratic surface """
    ncols = (order + 1)**2
    G = np.zeros((x.size, ncols))
    ij = itertools.product(range(order+1), range(order+1))
    for k, (i, j) in enumerate(ij):
        G[:, k] = x**i * y**j
    return G   

def Reward_Surface(opts):
    x=np.array([1,0,0,1,.5,0,.5])
    y=np.array([0,0,1,1,.5,.5,0])
    z=np.array([])
    a=opts.a
    b=opts.b
    for i in range(0,len(x)):
        z=np.append(z,(a*(x[i])**2)+(b*(y[i])**2))
    
    ordr=2
    G = poly_matrix(x, y, ordr)
    # Solve for np.dot(G, m) = z:
    m = np.linalg.lstsq(G, z,rcond=None)[0]
    nx, ny = 1000, 1000
    
    xx, yy = np.meshgrid(np.linspace(0, 1, nx),
                         np.linspace(0, 1, ny))
    GoG = poly_matrix(xx.ravel(), yy.ravel(), ordr)
    zz = np.reshape(np.dot(GoG, m), xx.shape)
#     this_dir, this_filename = os.path.split(__file__)
#     base_folder =this_dir
    with open('./Trial_Data/Reward_Data.npy','rb') as f:
        Data = np.load(f)
    X_Data=Data[:,0]
    Y_Data=Data[:,1]
    Z_Data=Data[:,2]

    GG = poly_matrix(X_Data, Y_Data, ordr)
# Solve for np.dot(G, m) = z:
    mm = np.linalg.lstsq(GG, Z_Data,rcond=None)[0]

    GoGG = poly_matrix(xx.ravel(), yy.ravel(), ordr)
    Reward_Ind = np.reshape(np.dot(GoGG, mm), xx.shape)[:,-1]
    return zz,Reward_Ind


In [ ]:
print(os.getcwd())

# RL_NECESSITIES

In [ ]:
# -*- coding: utf-8 -*-
"""
Created on Thu Aug 12 17:06:46 2021

@author: nbrow
"""
import numpy as np
from tensorflow.keras.optimizers import Adam # (if keras version 2.3.1)
from tensorflow.math import argmax
import tensorflow.keras as keras 
from tensorflow.keras import layers,models 
import os

os.environ["cuda_visible_devices"]="1"

class Agent():
    def __init__(self,env,opts,Increase,n_actions,input_dims,epsilon,
                 filename_save,filename_load,EX,EY):
        self.action_space = [i for i in range(n_actions)]
        self.n_actions=n_actions
        self.gamma = opts.gamma
        self.epsilon = epsilon
        self.EX=EX
        self.EY=EY
        self.env=env
        self.eps_dec = opts.epsilon_dec
        self.eps_min = opts.eps_end
        self.replace = opts.replace
        self.batch_size = opts.batch_size
        self.lr=opts.lr
        self.learn_step_counter = 0
        self.memory = ReplayBuffer(opts.mem_size, input_dims)
        self.q_eval = DuelingDeepQNetwork(self.EX*self.EY,Increase)
        self.q_next = DuelingDeepQNetwork(self.EX*self.EY,Increase)
#         this_dir, this_filename = os.path.split(__file__)
        self.checkpoint_file_save='C:/Users/dabni/Desktop/TO/RL_topopt_KSME2023-ksme2023/Top_Opt_RL/DQN/DAB/NN_Weights/'+filename_save+'_NN_weights'
        self.checkpoint_file_load='C:/Users/dabni/Desktop/TO/RL_topopt_KSME2023-ksme2023/Top_Opt_RL/DQN/DAB/NN_Weights/'+filename_load+'_NN_weights'
        self.q_eval.compile(optimizer=Adam(learning_rate=self.lr),
                            loss='mean_squared_error')
        # just a formality, won't optimize network
        self.q_next.compile(optimizer=Adam(learning_rate=self.lr),
                            loss='mean_squared_error')

    def store_transition(self, state, action, reward, new_state, done):
        self.memory.store_transition(state, action, reward, new_state, done)

    def choose_action(self, observation,load_checkpoint,Testing):
        self.action_space = [i for i in range(self.n_actions)]
        if np.random.random() < self.epsilon and not load_checkpoint and not Testing:
            Void=np.array(self.env.VoidCheck)
            BC=np.array(np.reshape(self.env.BC_state,(1,(self.EX*self.EY))))
            LC=np.array(np.reshape(self.env.LC_state,(1,(self.EX*self.EY))))
            Clear_List=np.where(Void==0)[0]
            BC_List=np.where(BC==1)[0]
            LC_List=np.where(LC==1)[0]
            self.action_space = [ele for ele in self.action_space if ele not in Clear_List]
            self.action_space = [ele for ele in self.action_space if ele not in BC_List]
            self.action_space = [ele for ele in self.action_space if ele not in LC_List]
            action = np.random.choice(self.action_space)
        else:
            state = observation
            state=state.reshape(-1,self.EX,self.EY,3)
            actions = self.q_eval.call(state)
            action=argmax(actions, axis=1).numpy()[0]

        return action

    def learn(self):

        if self.memory.mem_cntr < self.batch_size:
            Loss=.5
            return Loss

        if self.learn_step_counter % self.replace == 0 and self.learn_step_counter>0:
            self.q_next.set_weights(self.q_eval.get_weights())  

        states, actions, rewards, states_, dones = \
                                    self.memory.sample_buffer(self.batch_size)
        q_pred = self.q_eval(states)
        self.q_pred=q_pred
        q_next = self.q_next(states_)
        q_target = q_pred.numpy()
        max_actions = argmax(self.q_eval(states_), axis=1)
        # improve on my solution!
        for idx, terminal in enumerate(dones):
            #if terminal:
                #q_next[idx] = 0.0
            q_target[idx, actions[idx]] = rewards[idx] + \
                    self.gamma*q_next[idx, max_actions[idx]]*(1-int(dones[idx]))
        self.q_target = q_target
        Loss=np.subtract(q_target,q_pred.numpy())
        Loss=np.square(Loss)
        Loss=Loss.mean()
        self.q_eval.train_on_batch(states, q_target)

        self.epsilon = self.epsilon - self.eps_dec if self.epsilon > \
                        self.eps_min else self.eps_min 

        self.learn_step_counter += 1
        if self.learn_step_counter>5000:
            self.lr=2.5e-3
        if self.learn_step_counter>7500:
            self.lr=1e-3
        return Loss
        
    def save_models(self):
        print('... saving models ...')
        self.q_eval.save_weights(self.checkpoint_file_save)

    def load_models(self):
        print('... loading models ...')
        self.q_eval.load_weights(self.checkpoint_file_load).expect_partial()
        
class DuelingDeepQNetwork(keras.Model):
    def __init__(self, n_actions,Increase):
        super(DuelingDeepQNetwork, self).__init__()
        self.model = models.Sequential()

        self.model.add(layers.Conv2D(16,(3,3),padding='same',activation='relu'))
        self.model.add(layers.Conv2D(8,(3,3),padding='same',activation='relu'))
        self.model.add(layers.Conv2D(4,(3,3),padding='same',activation='relu'))
        self.model.add(layers.Conv2D(1,(3,3),padding='same',activation='relu'))
        self.model.add(layers.Flatten())
    
    def call(self, state):
        x = self.model(state)
    
        #V = self.model_V(x)
        #A = self.model_A(x)
        
        Q = x#V + (A - tf.math.reduce_mean(A, axis=1, keepdims=True))
        return Q
    
class ReplayBuffer():
    def __init__(self, max_size, input_shape):
        self.mem_size = max_size
        self.mem_cntr = 0
        self.state_memory = np.zeros((self.mem_size, *input_shape),
                                        dtype=np.float32)
        self.new_state_memory = np.zeros((self.mem_size, *input_shape),
                                        dtype=np.float32)
        self.action_memory = np.zeros(self.mem_size, dtype=np.int32)
        self.reward_memory = np.zeros(self.mem_size, dtype=np.float32)
        self.terminal_memory = np.zeros(self.mem_size, dtype=np.bool)

    def store_transition(self, state, action, reward, state_, done):
        index = self.mem_cntr % self.mem_size
        self.state_memory[index] = state
        self.new_state_memory[index] = state_
        self.action_memory[index] = action
        self.reward_memory[index] = reward
        self.terminal_memory[index] = done
        self.mem_cntr += 1

    def sample_buffer(self, batch_size):
        max_mem = min(self.mem_cntr, self.mem_size)
        batch = np.random.choice(max_mem, batch_size, replace=False)
        states = self.state_memory[batch]
        new_states = self.new_state_memory[batch]
        actions = self.action_memory[batch]
        rewards = self.reward_memory[batch]
        dones = self.terminal_memory[batch]

        return states, actions, rewards, new_states, dones


# PLOT_LAYOUT

In [ ]:
'''
Plot the layout of the structure
input: 1. file_name (string, the name of the json file, e.g. 'App_Data.json')
       2. number of elements (tuple, e.g. (24,24))
       3. save_name (string, the name of the saved image, e.g. 'result.jpg')
'''
import numpy as np
import json
import matplotlib.pyplot as plt

def plot_layout(file_name = 'App_Data.json', number_of_elements = (24, 24), save_name = 'result.jpg'):
    with open(file_name) as file:
        datas = json.load(file)
        aa = datas["Topology"]
    
    aa_sq = np.array(aa).squeeze().reshape(number_of_elements[0], number_of_elements[1])
    aa_num = aa_sq.astype(np.float64)
    aa_num = np.int64(aa_num)
    
    
        
    fig, ax = plt.subplots()
    ax.matshow(aa_num, cmap=plt.cm.Blues)
#     for i in range(number_of_elements[0]):
#         for j in range(number_of_elements[1]):
#             c = aa_num[j,i]
#             ax.text(i, j, str(c), va='center', ha='center')
    plt.savefig(save_name, dpi=300)
    plt.show()
    

# MAIN

In [ ]:
# -*- coding: utf-8 -*-
"""
Created on Fri Apr  2 09:34:14 2021

@author: nbrow
@modified for primary mirror application 
"""

''' Nathan Brown 
Main Function for the Reinforcement Learning Based Topology Optimization Solver using Double Q-Learning'''
import os, sys
os.environ['TF_CPP_MIN_LOG_LEVEL'] = '3' 
import numpy as np
import time
import json
import math
sys.path.append(os.path.dirname(os.path.abspath(os.path.dirname(os.path.abspath(os.path.dirname("TOPRL"))))))
# import FEA_SOLVER_GENERAL
# from Top_Opt_RL.DQN.FEA_SOLVER_GENERAL import *
# from FEA_SOLVER_GENERAL import *
# from Top_Opt_RL.DQN.FEA_SOLVER_GENERAL import FEA_SOLVER_GENERAL

# from Top_Opt_RL.DQN.opts import parse_opts
# from Top_Opt_RL.DQN.TopOpt_Env_Functions import TopOpt_Gen, Prog_Refine_Act,User_Inputs,App_Inputs, Testing_Inputs, Testing_Info, Min_Dist_Calc  
# from Top_Opt_RL.DQN.Matrix_Transforms import obs_flip, action_flip, Mesh_Transform, Mesh_Triming 
# from Top_Opt_RL.DQN.RL_Necessities import Agent 
def plot_learning_curve(x, scores, figure_file):
    import matplotlib.pyplot as plt
    running_avg = np.zeros(len(scores))
    for i in range(len(running_avg)):
        running_avg[i] = np.mean(scores[max(0, i-50):(i+1)])
    plt.plot(x, running_avg)
    plt.title('Running average of previous 100 scores')
    plt.xlabel('Episodes')
    plt.ylabel(' Average Reward')
    plt.savefig(figure_file)
def Data_History(score_history, per_history, succ_history, Loss_history, Total_Loss, score, Main_EX, Main_EY,i):

    Loss_history.append(Total_Loss)
    avg_Loss=np.mean(Loss_history[-50:])
    score_history.append(score)
    avg_score = np.mean(score_history[-50:])
    Succ_Steps=list(env.VoidCheck).count(0)
    succ_history.append(Succ_Steps)

    avg_succ = np.mean(succ_history[-50:])
    Percent_Succ=Succ_Steps/(Main_EX*Main_EY)
    per_history.append(Percent_Succ)
    avg_percent=np.mean(per_history[-50:])
    return score_history,per_history,succ_history,Loss_history,Succ_Steps,Percent_Succ,avg_succ,avg_score,avg_Loss,avg_percent

def TopOpt_Designing(User_Conditions, opts, envs): #,my_call_back_functions):
    Time_Trial = opts.Time_Trial
    if opts.Progressive_Refinement:
        agent_primer= Agent(envs.env_primer,opts,Increase=False,filename_save=opts.filename_save+str(opts.PR_EX)+'by'+str(opts.PR_EY),
                            filename_load=opts.filename_load,EX=opts.PR_EX,EY=opts.PR_EY, n_actions=opts.PR_EX*opts.PR_EY,
                            epsilon=0,input_dims=[opts.PR_EX,opts.PR_EY,3])
                            
        agent_primer2= Agent(envs.env_primer2,opts,Increase=False,filename_save=opts.filename_save+str(opts.PR2_EX)+'by'+str(opts.PR2_EY),
                            filename_load=opts.filename_load,EX=opts.PR2_EX,EY=opts.PR2_EY, n_actions=opts.PR2_EX*opts.PR2_EY, 
                            epsilon=0,input_dims=[opts.PR2_EX,opts.PR2_EY,3])
        agent_primer.load_models()
        agent_primer2.load_models()
    
    agent = Agent(envs.env,opts,Increase=False,filename_save=opts.filename_save+str(opts.Main_EX)+'by'+str(opts.Main_EY),
                  filename_load=opts.filename_load,EX=opts.Main_EX,EY=opts.Main_EY, n_actions=opts.Main_EX*opts.Main_EY, 
                  epsilon=1.0, input_dims=[opts.Main_EX,opts.Main_EY,3])
    if opts.Load_Checkpoints: agent.load_models()    
    figure_file = 'plots/' + opts.filename_save +'_reward.png'    
    best_score = envs.env.reward_range[0]    
    score_history ,per_history,succ_history,Loss_history= [],[],[],[]
    
    if not opts.Load_Checkpoints:
        from pandas import DataFrame 
        TrialData=DataFrame(columns=['Episode','Reward','Successfull Steps','Percent Successful','Avg Loss','SDEV','Epsilon','Time'])
    envs.env.reset_conditions()
    if opts.From_App:  opts.n_games=1
    for i in range(opts.n_games):
        Testing = False #Used to render the environment and track learning of the agent 
        if opts.Load_Checkpoints:
            'If the user wants to test the agent, the user will be prompted to input BC and LC elements'
            if opts.From_App:  App_Inputs(envs.env,envs.env_primer,envs.env_primer2,opts,User_Conditions)

            else:  User_Inputs(envs.env,opts)

        done = False
        score = 0    
        if i%10==0 and i>=100:
            Testing=True
            if i%200==0:
                'Every 200 episodes, a special BC/LC will be used for monitoring purposes'
                Testing_Inputs(envs.env,opts)
                print('--------Testing Run------')
        envs.env.VoidCheck=list(np.ones((1,envs.env.EX*envs.env.EY))[0])
        if Time_Trial:     Start_Time_Trial=time.perf_counter()
        observation = envs.env.reset()
        print(envs.env)
        if opts.Progressive_Refinement:
            ''' Set Up to Complete 3 Iterations of Progressive Refinement'''
            #Progressive Refinement #1 Going from Smallest to Intermediate Mesh Size
            envs.env_primer.VoidCheck=list(np.ones((1,envs.env_primer.EX*envs.env_primer.EY))[0])
            Prog_Refine_Act(agent_primer,envs.env,envs.env_primer,opts.Load_Checkpoints,Testing,opts,opts.PR_EX,opts.PR_EY,Time_Trial,opts.From_App,FEA_Skip=1)
            #Progressive Refinement #2 Going for Intermediate to Final Mesh Size
            envs.env_primer2.VoidCheck=Mesh_Transform(opts.PR_EX,opts.PR_EY,opts.PR2_EX,opts.PR2_EY,envs.env_primer.VoidCheck)
            if opts.From_App:
                del agent_primer
            Prog_Refine_Act(agent_primer2,envs.env,envs.env_primer2,opts.Load_Checkpoints,Testing,opts,opts.PR2_EX,opts.PR2_EY,Time_Trial,opts.From_App,FEA_Skip=1)
            #This outcome will now be used as the final mesh Size 
            envs.env.VoidCheck=Mesh_Transform(opts.PR2_EX,opts.PR2_EY,opts.Main_EX,opts.Main_EY,envs.env_primer2.VoidCheck)
            if opts.From_App:
                del agent_primer2
            #Removed_Num=Mesh_Triming(env_primer,PR_EX,PR_EY)
            #Uncomment the above line if you want to incorporate mesh trimming

            observation[:,:,0]=np.reshape(FEASolve(envs.env.VoidCheck,opts.Lx,opts.Ly,opts.Main_EX,opts.Main_EY,envs.env.LC_Nodes,envs.env.Load_Directions,envs.env.BC_Nodes,Stress=True)[3],(opts.Main_EX,opts.Main_EY))
        observation_v, observation_h,observation_vh=obs_flip(observation,opts.Main_EX,opts.Main_EY)
        Last_Reward=0
        while not done:
            if i%1000==0 and i>=1: #Every 1000 iterations, show the activation maps
                from keract import get_activations, display_activations 
                activations = get_activations(agent.q_eval.model, observation.reshape(-1,opts.Main_EX,opts.Main_EY,3))
                display_activations(activations, save=False)
            action = agent.choose_action(observation,opts.Load_Checkpoints,Testing)
            observation_, reward, done, It= envs.env.step(action,observation,Last_Reward,opts.Load_Checkpoints,envs.env,FEA_Skip=1,PR=False)
            if not opts.Load_Checkpoints:
                observation_v_,observation_h_,observation_vh_=obs_flip(observation_,opts.Main_EX,opts.Main_EY)
                action_v,action_h,action_vh=action_flip(action,opts.Main_EX,opts.Main_EY)
                agent.store_transition(observation,action,reward,observation_,done)
                agent.store_transition(observation_v,action_v,reward,observation_v_,done)
                agent.store_transition(observation_h,action_h,reward,observation_h_,done)
                agent.store_transition(observation_vh,action_vh,reward,observation_vh_,done)
            score += reward
            App_Plot=Testing_Info(envs.env,envs.env_primer,envs.env_primer2,opts,score,opts.Progressive_Refinement,opts.From_App,Fixed=True)
            # _=[fn(App_Plot) for fn in my_call_back_functions]
            Last_Reward=reward
            if Testing and not Time_Trial:
                envs.env.render()
                print('Current Score: '+str(round(score,3)))
            observation = observation_
            if not opts.Load_Checkpoints:
                observation_v=observation_v_
                observation_h=observation_h_
                observation_vh=observation_vh_
            if opts.Load_Checkpoints and not Time_Trial:   envs.env.render()
        App_Plot=Testing_Info(envs.env,envs.env_primer,envs.env_primer2,opts,score,opts.Progressive_Refinement,opts.From_App,Fixed=True)
        # _=[fn(App_Plot) for fn in my_call_back_functions]
        return App_Plot        
        toc=time.perf_counter()

        if Time_Trial and not opts.From_App:
            print('It took '+str(round(toc-Start_Time_Trial,1))+' seconds to complete this time trial.')    

        App_Plot=Testing_Info(envs.env,envs.env_primer,envs.env_primer2,opts,score,opts.Progressive_Refinement,opts.From_App,Fixed=True)
class EnviromentsRL:
    def __init__(self, opts):
        if opts.Load_Checkpoints:
            SC=opts.SC
            if opts.VF_S==0 and opts.From_App: #If the user wants to set a final volume fraction, set the intermediate volume fractions accordingly
                Vol_Frac_2=opts.Vol_Frac_2
                Vol_Frac_1=opts.Vol_Frac_1
                Vol_Frac_3=opts.Vol_Frac_3 
        else:
            Vol_Frac_3=opts.Vol_Frac_3
            Vol_Frac_1=opts.Vol_Frac_1
            Vol_Frac_2=opts.Vol_Frac_2
        self.env = TopOpt_Gen(opts.Main_EX,opts.Main_EY,Vol_Frac_3,SC,opts)
        self.env_primer= TopOpt_Gen(opts.PR_EX,opts.PR_EY,Vol_Frac_1,SC,opts)
        self.env_primer2=TopOpt_Gen(opts.PR2_EX,opts.PR2_EY,Vol_Frac_2,SC,opts)



In [ ]:
''' 
Class for Topology Optimization Options
# MAIN / PR2 한 만큼 각져짐 
'''
class Top_Options:
    def __init__(self, Main_EX=48, Main_EY=48, PR2_EX=24, PR2_EY=24, PR_EX=3, PR_EY=3, Lx=1, Ly=1, Eta=2, a=5, b=5, replace=100, epsilon_dec=3.5e-4, eps_end=0.01, mem_size=30000, n_games=50000, batch_size=128, lr=5e-3, gamma=0.1, Vol_Frac_1=0.7, Vol_Frac_2=0.5, Vol_Frac_3=0.25, SC=10, P_Norm=10, filename_save='DDQN_TopOpt_Generalized_CNN_4L', filename_load='DDQN_TopOpt_Generalized_CNN_4L_6by6', Progressive_Refinement=True, LC=False, Load_Checkpoints=True, VF_S=0, Min_Dist=0, Time_Trial=True, configfile='config.json', From_App=True, base_folder="."):
        self.Main_EX = Main_EX  # Number of X Elements for Larger Environment
        self.Main_EY = Main_EY  # Number of Y Elements for Larger Environment
        self.PR2_EX = PR2_EX  # Number of X Elements for Second Environment used in Case of Progressive Refinement
        self.PR2_EY = PR2_EY  # Number of Y Elements for Second Environment used in Case of Progressive Refinement
        self.PR_EX = PR_EX  # Number of X Elements for Smaller Environment used in Case of Progressive Refinement
        self.PR_EY = PR_EY  # Number of Y Elements for Smaller Environment used in Case of Progressive Refinement
        self.Lx = Lx  # Length of the Structure in the X Direction
        self.Ly = Ly  # Length of the Structure in the Y Direction
        self.Eta = Eta  # Used for dynamic adjusting reward function. Larger eta means less prevalence given towards changes between current and previous reward. Recommend using [2,4]
        self.a = a  # X Coefficient of the Quadratic Reward Surface
        self.b = b  # Y Coefficient of the Quadratic Reward Surface
        self.replace = replace  # Number of iterations between switching the weights from the active network to the target network
        self.epsilon_dec = epsilon_dec  # Iterative decay amount of the epsilon value used for exploration/explotation
        self.eps_end = eps_end  # Smallest Allowable Epsilon value to be used for exploration/explotation
        self.mem_size = mem_size  # Size of the Replay Buffer
        self.n_games = n_games  # Maximum Number of Training Episodes Conducted
        self.batch_size = batch_size  # Batch Size that will be taken from the Replay Buffer per training episode
        self.lr = lr  # Starting Learning Rate for the Network
        self.gamma = gamma  # Discount Factor for Future Rewards
        self.Vol_Frac_1 = Vol_Frac_1  # Volume Fraction during first progressive refinement
        self.Vol_Frac_2 = Vol_Frac_2  # Final Volume Fraction
        self.Vol_Frac_3 = Vol_Frac_3  # Final Volume Fraction
        self.SC = SC  # Stress constraint, between 0 and 2
        self.P_Norm = P_Norm  # Smoothing Parameter for P-Norm Global Stress calculation
        self.filename_save = filename_save  # When training, what name would you like your weights, and figure saved as
        self.filename_load = filename_load  # When testing, what name is your NN weights saved under
        self.Progressive_Refinement = Progressive_Refinement
        self.LC = LC # type in loading conditions manually
        self.Load_Checkpoints = Load_Checkpoints
        self.VF_S = VF_S # Use vol fraction constraint [0] or stress constraint [1]
        self.Min_Dist = Min_Dist # The 0 value serves as a place holder to represent the minimum distance between the bounded and loaded elements in a given load case
        self.Time_Trial = Time_Trial # Perform Time Trial
        self.configfile = configfile # name of config file. 
        self.From_App = From_App # True if being called by an external app. Not sure this is needed. "
        self.base_folder = base_folder # Folder where to find saved files. Helpful if not running the app from the main folder. 



In [ ]:
opts = Top_Options()
opts.configfile = ".json"
User_Conditions = json.load(open(opts.configfile) ) if opts.From_App else None  
envs = EnviromentsRL(opts)  
App_Plot=TopOpt_Designing(User_Conditions,opts, envs)
json.dump( App_Plot, open( "App_Data.json", 'w' ) )
plot_layout(file_name='App_Data.json', number_of_elements=(opts.Main_EX, opts.Main_EY), save_name='App_Plot.png')
